In [ ]:
from quantopian.research.experimental import continuous_future, history

import pandas as pd

### 5限月分の終値を取得

In [ ]:
fut = 'SY'
start = '2015-1-1'
end='2018-5-13'

df = history(
    [continuous_future(fut, offset=i,  adjustment=None) for i in range(5)],
    fields='price', 
    frequency='daily', 
    start=start, 
    end=end
)

df.columns = ['f{}'.format(i) for i in range(1, 6)]

In [ ]:
df.head()


### 1つ前の限月との比率

In [ ]:
ratio_df = pd.DataFrame([(df.iloc[:, i] / df.iloc[:, i+1]) - 1 for i in range(len(df.columns) - 1)]).T
ratio_df.columns = ['f{}-f{}'.format(i, i + 1) for i in range(1, len(ratio_df.columns) + 1)]

In [ ]:
ratio_df.head()


In [ ]:
df.pct_change(axis=1).head()

In [ ]:
ratio_df.plot()

### 1限月と2限月の比率と他の比率(1つ前の限月の比率)の差

In [ ]:
ratio_diff_df = pd.DataFrame([abs(ratio_df.iloc[:, 0] + ratio_df.iloc[:, i] * -1) for i in range(1, len(ratio_df.columns))]).T
ratio_diff_df.columns = ratio_df.columns[1:]

In [ ]:
ratio_diff_df.plot()

In [ ]:
ratio_diff_df.hist()

In [ ]:
ratio_diff_df.describe()

### トレード
* 1-2限月の比率と対象の限月の比率の符号が逆ならトレード
* threshold: 比率の差が大きければトレードをするというフィルタをかける
* hold: 保有日数

In [ ]:
def make_trade_df(pair, threshold, hold=1):
    trade_df = ratio_df[['f1-f2', pair]].copy()
    trade_df['long_short'] = 0
    trade_df.loc[(trade_df.loc[:, 'f1-f2'] < 0) & (trade_df.loc[:, pair] > 0), 'long_short'] = -1
    trade_df.loc[(trade_df.loc[:, 'f1-f2'] > 0) & (trade_df.loc[:, pair] < 0), 'long_short'] = 1
    trade_df['pct_change'] = (trade_df[pair] + 1).pct_change(hold).shift(-hold)
    trade_df['returns'] = (trade_df['long_short'] * trade_df['pct_change']).cumsum()
    trade_df['signal'] = ratio_diff_df[pair] > threshold
    trade_df['filtered_returns'] = (trade_df['long_short'] * trade_df['pct_change'] * trade_df['signal']).cumsum()
    return trade_df

#### 4-5限月をしきい値を比率の差の四分位に設定

In [ ]:
trade_df = make_trade_df('f4-f5', ratio_diff_df['f4-f5'].quantile(0.25))

In [ ]:
trade_df.head()

In [ ]:
trade_df[['returns', 'filtered_returns']].plot()

#### 保有日数を5日間に設定

In [ ]:
make_trade_df('f4-f5', ratio_diff_df['f4-f5'].quantile(0.25), hold=5)[['returns', 'filtered_returns']].plot()

In [ ]:
df = history(
    [continuous_future(fut, offset=i,  adjustment=None) for i in range(5)],
    fields='volume', 
    frequency='daily', 
    start=start, 
    end=end
)


In [ ]:
df.iloc[:,2:].plot()

In [ ]:
df = history(
    [continuous_future(fut, offset=i,  adjustment=None) for i in range(5)],
    fields='volume', 
    frequency='minute', 
    start="2018-5-2", 
    end="2018-5-2",
)


In [ ]:
df.plot()